In [ ]:
print("This is a test")

In [ ]:
from datetime import datetime
from hogwarts.auth.vault.vault_client import VaultClient

import requests

vault_client = VaultClient()
vault_client.login()

test = vault_client.get_personal_secret("Test-Secret")

testUser = test["test_username"]
testValue = test["test_value"]

print("Testing secrets")
print("Username: " + testUser)
print("Value: " + testValue)

#print("g_sqlDatabase: " + g_sqlDatabase)


In [ ]:
commitLocation = "https://api.github.com/repos/jmiller-gc/Git_Test/commits?page=1&per_page=1"

trackerSecret = vault_client.get_personal_secret("Tracker")

lastCommit = trackerSecret["DOMAIN_LIST_LAST_COMMIT_DATE"]
forceUpdate = trackerSecret["FORCE_UPDATE_DOMAIN_LIST"]

# Get last commit date from secrets and convert it to datetime.
if (lastCommit == "None"):
    lastCommitDate = datetime(1, 1, 1, 0, 0, 0)
else:
    lastCommitDate = datetime.strptime(lastCommit, "%Y-%m-%d %H:%M:%S")

# Get datetime of most current commit
with requests.get(commitLocation) as response:
    response.raise_for_status()
    currentCommit = datetime.strptime(response.json()[0]['commit']['committer']['date'], '%Y-%m-%dT%H:%M:%SZ')

print("Last commit processed: " + lastCommit)
print("Most recent update available: " + currentCommit.strftime("%Y-%m-%d %H:%M:%S"))

if ((forceUpdate) or (lastCommitDate < currentCommit)):
    if (forceUpdate):
        print("Domain list updated due to override")
    else:
        print("Domain list updated due to new commit")

    vault_client.set_personal_secret("Tracker", {"USERNAME" : trackerSecret["USERNAME"],
                                                 "PASSWORD" : trackerSecret["PASSWORD"],
                                                 "DOMAIN_LIST_LAST_COMMIT_DATE" : currentCommit.strftime("%Y-%m-%d %H:%M:%S"),
                                                 "FORCE_UPDATE_DOMAIN_LIST" : 0})